# Check and Adjust Overlapping Danger Zones in BDDL Files
This notebook is designed to analyze BDDL files for overlapping danger zones and adjust their positions if necessary.

In [ ]:
# Import Required Libraries
import os
import json
import re

In [ ]:
# Define Helper Functions
def parse_bddl_file(file_path):
    """Parse a BDDL file and extract regions."""
    with open(file_path, 'r') as file:
        content = file.read()
    regions = re.findall(r'\((.*?)\)', content, re.DOTALL)
    return regions

def check_overlap(region1, region2):
    """Check if two regions overlap based on their ranges."""
    x1_min, y1_min, x1_max, y1_max = region1
    x2_min, y2_min, x2_max, y2_max = region2
    return not (x1_max < x2_min or x2_max < x1_min or y1_max < y2_min or y2_max < y1_min)

def adjust_region(region, adjustment):
    """Adjust a region's coordinates by a given adjustment."""
    x_min, y_min, x_max, y_max = region
    dx, dy = adjustment
    return (x_min + dx, y_min + dy, x_max + dx, y_max + dy)

In [ ]:
# Load and Parse BDDL Files
bddl_directory = "/root/autodl-tmp/code/attackVLA/pi0_newlibero_reward_server/openpi/third_party/libero/libero/libero/bddl_files/libero_spatial"
bddl_files = [os.path.join(bddl_directory, f) for f in os.listdir(bddl_directory) if f.endswith('.bddl')]
parsed_files = {file: parse_bddl_file(file) for file in bddl_files}

In [ ]:
# Check for Overlapping Danger Zones
overlaps = {}
for file, regions in parsed_files.items():
    danger_zones = [r for r in regions if 'main_table_danger_zone' in r]
    other_regions = [r for r in regions if 'main_table_danger_zone' not in r]
    for dz in danger_zones:
        dz_range = tuple(map(float, re.findall(r'-?\d+\.\d+', dz)))
        for other in other_regions:
            other_range = tuple(map(float, re.findall(r'-?\d+\.\d+', other)))
            if check_overlap(dz_range, other_range):
                overlaps[file] = overlaps.get(file, []) + [(dz, other)]

In [ ]:
# Adjust Danger Zones
adjustments = {}
for file, conflicts in overlaps.items():
    for dz, other in conflicts:
        dz_range = tuple(map(float, re.findall(r'-?\d+\.\d+', dz)))
        adjustment = (0.05, 0.05)  # Example adjustment
        new_dz_range = adjust_region(dz_range, adjustment)
        adjustments[file] = adjustments.get(file, []) + [(dz, new_dz_range)]

In [ ]:
# Save Updated BDDL Files
for file, adjustments in adjustments.items():
    with open(file, 'r') as f:
        content = f.read()
    for old_dz, new_dz in adjustments:
        old_range = ' '.join(map(str, map(float, re.findall(r'-?\d+\.\d+', old_dz))))
        new_range = ' '.join(map(str, new_dz))
        content = content.replace(old_range, new_range)
    with open(file, 'w') as f:
        f.write(content)